# Problem Statement

TO BE ADDED AFTERWARDS 

# Data Sources

# Data Transformations + EDA

# Modelling

# Trading Simulation

# Automation ( from here onwards it should be moved to local)

# Bonus points